In [36]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action="ignore", category=Warning)
import gc
gc.collect()

6864

In [38]:
data_frame_copy19 = pd.read_csv("../PROJECTS/FIFA/players_19.csv")
data_frame_copy20 = pd.read_csv("../PROJECTS/FIFA/players_20.csv")

In [39]:
data_frame19 = data_frame_copy19
data_frame19.head(1)

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",94,94,77000000.0,400000.0,33,...,54+4,54+4,54+4,61+4,20+4,https://cdn.sofifa.net/players/020/801/19_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png


# Create long-wide and mixed dataframe formats

In [41]:
data_frame = data_frame_copy19[["short_name","nationality_name", "age", "club_name"]]
data_frame_mixed = data_frame.set_index(keys = ["nationality_name", "short_name"]).sort_index()
wide_format = data_frame_mixed.loc[ ["Poland", "Sweden", "Greece"] , : ].sample(random_state = 22, n = 12).sort_index(ascending = False)
wide_format.reset_index(level="nationality_name", drop = False, inplace = True)

In [45]:
multiindex = ["nationality_name", "short_name", "feature"]
long_format = pd.melt(
    frame = data_frame_wide.reset_index(drop = False),
    id_vars= multiindex[:-1],
    value_vars = ["age", "club_name"],
    var_name= "feature",
    value_name="value"
).set_index(multiindex).sort_index(ascending = False)

# Creating multiindex for rows, for columns and both

In [48]:
multilevel_columns = data_frame.sample(n = 5, random_state=21).set_index(multiindex[:-1]).transpose()

In [50]:
multilevel_rows = data_frame.sample(n = 5, random_state=21).set_index(multiindex[:-1])

In [52]:
multiindex.remove("feature")
multiindex.remove("short_name")
multiindex.extend(["weight_kg", "height_cm"])

In [54]:
names1 = list(data_frame_copy19["short_name"])
names2 = list(data_frame_copy20["short_name"])
names = list(set(names1) & set(names2))[:5]

In [56]:
temp1 = data_frame_copy19.set_index("short_name").loc[names, multiindex ]
temp2 = data_frame_copy20.set_index("short_name").loc[names, multiindex ]
temp1["year"], temp2["year"] = 2019, 2020

In [58]:
multilevel_rows_cols = pd.concat(axis=0, objs=[temp1, temp2]).reset_index(drop = False)

In [60]:
multilevel_rows_cols = pd.DataFrame(multilevel_rows_cols.set_index(keys = ["nationality_name", "short_name", "year"]).sort_index().unstack())

In [79]:
default_range_idx = data_frame_copy19.loc[ : , multiindex].head(5)

# Results - at this point we have all possible DataFrame shapes (5)
* The shapes are :
singlelevel rows and cols, multilevel rows and cols, singlelevel rows and multilevel cols, multilevel rows and singlelevel cols, no row index + single level cols


* Long format (one row one observation all attr in separated row index or column - feature)
    * singlelevel cols + singlelevel rows
    * single cols + multilevel rows
    * without rows index
* Wide format (one row one entity with all attr in one row)
    * singlelevel rows + singlelevel cols
    * singlelevel rows + multilevel cols
    * without rows index
* Multilevel format
    * multilevel cols and rows
    * multilevel cols + single rows (considered also as wide format)
    * multilevel rows + single cols (considered also as long format)

In [63]:
wide_format.head(3)

,nationality_name,age,club_name
short_name,,,
H. Andersson,Sweden,19,Trelleborgs FF
D. Boo Wiklander,Sweden,33,IFK Göteborg
A. Erlingmark,Sweden,20,IFK Göteborg


In [65]:
long_format.head(3)

value
nationality_name short_name       feature                  
Sweden           H. Andersson     club_name  Trelleborgs FF
                                  age                    19
                 D. Boo Wiklander club_name    IFK Göteborg

In [67]:
multilevel_columns.head(2)

nationality_name,Italy,Chile,Saudi Arabia,France,Portugal
short_name,L. Scaglia,F. Manzano,M. Salem,N. Diallo,Bruno Varela
age,31,24,32,22,23
club_name,Parma,Club Deportivo Palestino,Al Shabab,Stade Rennais FC,SL Benfica


In [69]:
multilevel_rows.head(2)

,,age,club_name
nationality_name,short_name,,
Italy,L. Scaglia,31,Parma
Chile,F. Manzano,24,Club Deportivo Palestino


In [83]:
default_range_idx.head(2)

,nationality_name,weight_kg,height_cm
0,Portugal,83,187
1,Argentina,72,170


# Stacking and unstacking